# Date Mining -01- Import

### Setup

In [1]:
import numpy as np
import pandas as pd

from IPython.display import display, Markdown
pd.set_option("max_colwidth", 1000)

import os,re

In [2]:
for d in ['data','output']:
    os.makedirs(d, exist_ok=True)

In [3]:
url = "https://setu-datamining2.github.io/live/topics/21-Assignments/03-Mining_Dates/files/public.csv"

if os.path.isfile("data/public.csv"):
    print("Using local copy ..")
else:
    print("Downloading ..")
    df = pd.read_csv(url)
    df.to_csv("data/public.csv", index=False)

Using local copy ..


### Dataset

In [4]:
df = pd.read_csv("data/public.csv")
print(df.shape)
df.head(10)

(715, 2)


,Code,Raw
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester."
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it."
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward."
3,1039963589,"October 7, 01 [report_end]"
4,1048901075,"July, 4, 01 Primary Care Doctor:"
5,1054311047,)and 8mo in 2009
6,1054668034,")HTN, hypercholesterolemia, DM, sleep apnea,, nephrolithiasis. chronic renal impairment, DVT since July 1977 on enoxaparin."
7,1082469285,"Septeber, 10, 70 CPT Code: 90792: With medical services"
8,1125769793,"Since 10/2014: Fatigued, more forgetful, impaired dexterity on her left hand. MRI reveals an approximately 4.2cm x 3.3cm x 2.5cm right parietal enhancing mass with surrounding edema"
9,1148116416,24 yo right handed woman with history of large right frontal mass s/p resection 11/3/1985 who had recent urgent R cranial wound revision and placement of L EVD for declining vision and increased drainage from craniotomy incision site and possible infection. She has a hx of secondary mania related to psychosis and manipulation of her right frontal lobe.


### Import/Clean
Algorithm
* Clean data first to reduce number of regexs needed
* Build sequence of regexs (Starting with most restrictive)
* For each regex, numbered 1,2,3,...
 * Find rows that match
 * Extract day month and year
 * Update df

In [5]:
def clean(s):
    return s

In [6]:
df['Data'] = df.Raw.apply(clean)
df['Iter'] = 0
df['Day'] = 0
df['Month'] = 0
df['Year'] = 0
df['Date'] = 0

### Helper functions

In [7]:
month_map = {
    'Jan': '01', 'January': '01',
    'Feb': '02', 'February': '02',
    'Mar': '03', 'March': '03',
    'Apr': '04', 'April': '04',
    'May': '05',
    'Jun': '06', 'June': '06',
    'Jul': '07', 'July': '07',
    'Aug': '08', 'August': '08',
    'Sep': '09', 'September': '09',
    'Oct': '10', 'October': '10',
    'Nov': '11', 'November': '11',
    'Dec': '12', 'December': '12'
}

In [8]:
from fuzzywuzzy import process

# Function to convert word months to numbers
def convert_word_month_to_number(month_str):
    # Check if the month string is in the month map
    if month_str in month_map:
        return month_map[month_str]
    else:
        # If not, find the closest match using fuzzy string matching
        closest_match = process.extractOne(month_str, month_map.keys())[0]
        return month_map[closest_match]

In [9]:
def info(n=None, show_unmatched=True, nrow=5):
    "Helper function to display results of a match and some rows not matched so far."
    
    n = df.Iter.max() if n is None else n
    for nn in ([n,0] if n!=0 and show_unmatched else [n]):
        display(Markdown("**Numer of rows with iter=%s: %s**" % (nn,len(df[df.Iter==nn]))))
        display(df.loc[df.Iter==nn,["Code","Data", "Iter", "Day", "Month", "Year"]].head(nrow))

In [10]:
# Function to convert two-digit years to four-digit years
def convert_to_four_digit_year(year):
    try:
        year = int(year)
        if year < 20:
            return 2000 + year
        elif year < 100:
            return 1900 + year
        else:
            return year
    except ValueError:
        return np.nan  # Return NaN for non-numeric values

In [11]:
def verify(df_tmp):
    # Convert 'Year' column to int if it contains numeric values
    if pd.api.types.is_numeric_dtype(df_tmp['Year']):
        df_tmp['Year'] = pd.to_numeric(df_tmp['Year'], errors='coerce')
    
    # Convert two-digit years to four-digit years
    df_tmp['Year'] = df_tmp['Year'].apply(convert_to_four_digit_year)
    
    # If 'Day' column is present, fill missing values with 1
    if 'Day' in df_tmp.columns:
        df_tmp['Day'].fillna(1, inplace=True)
    else:
        df_tmp['Day'] = 1
    
    # If 'Month' column is present, fill missing values with 1 (January)
    if 'Month' in df_tmp.columns:
        df_tmp['Month'].fillna(1, inplace=True)
    else:
        df_tmp['Month'] = 1
    
    return df_tmp

### 'mm/dd/yyyy'

In [12]:
iter = 0
columns = ['Month', 'Day', 'Year']

In [13]:
# Regular expression for numeric dates with inconsistent field separators
regex_numeric_dates = r"(\d{1,2})[-/](\d{1,2})[-/](\d{2,4})"


In [14]:
# Apply the regex to extract date components
df_numeric_dates = df[df.Iter == 0].Data.str.extract(regex_numeric_dates)
df_numeric_dates.dropna(inplace=True)
df_numeric_dates.columns = columns

In [15]:
print(f"Number of rows matched = {df_numeric_dates.shape[0]}")
df_numeric_dates.head()

Number of rows matched = 124


,Month,Day,Year
9,11,3,1985
10,4,19,91
14,07,29,1994
16,6,10,72
18,6,18,85


In [16]:
# Verify date components
df_numeric_dates = verify(df_numeric_dates)

In [17]:
# Convert 'Year' column to numeric
df_numeric_dates['Year'] = pd.to_numeric(df_numeric_dates['Year'], errors='coerce')

In [18]:
# Filter rows where 'Year' is not NaN and greater than 0
criteria = (df.Iter == 0) & (df_numeric_dates['Year'].notna()) & (df_numeric_dates['Year'] > 0)
df.loc[criteria, columns] = df_numeric_dates[columns]
df.loc[criteria, 'Iter'] = iter + 1

info()

**Numer of rows with iter=1: 124**

,Code,Data,Iter,Day,Month,Year
9,1148116416,24 yo right handed woman with history of large right frontal mass s/p resection 11/3/1985 who had recent urgent R cranial wound revision and placement of L EVD for declining vision and increased drainage from craniotomy incision site and possible infection. She has a hx of secondary mania related to psychosis and manipulation of her right frontal lobe.,1,3,11,1985
10,1157934136,4/19/91 Communication with referring physician?: Not Done,1,19,4,1991
14,1191233809,07/29/1994 CPT code: 99203,1,29,07,1994
16,1218956332,6/10/72 SOS-10 Total Score:,1,10,6,1972
18,1220889324,6/18/85 Primary Care Doctor:,1,18,6,1985


**Numer of rows with iter=0: 591**

,Code,Data,Iter,Day,Month,Year
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",0,0,0,0
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",0,0,0,0
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.",0,0,0,0
3,1039963589,"October 7, 01 [report_end]",0,0,0,0
4,1048901075,"July, 4, 01 Primary Care Doctor:",0,0,0,0


In [19]:
# Regular expression for inconsistent month names and separators
regex_inconsistent_month = r"(\w{3,})[-\s.]*([0-9]{1,2})[-,\s]+(\d{2,4})"

In [20]:
# Apply the regex to extract date components
df_inconsistent_month = df[df.Iter == 0].Data.str.extract(regex_inconsistent_month)
df_inconsistent_month.dropna(inplace=True)
df_inconsistent_month.columns = columns

In [21]:
# Apply the conversion function to the 'Month' column
df_inconsistent_month['Month'] = df_inconsistent_month['Month'].apply(convert_word_month_to_number)

In [22]:
print(f"Number of rows matched = {df_inconsistent_month.shape[0]}")
df_inconsistent_month.head()

Number of rows matched = 119


,Month,Day,Year
3,10,7,01
26,06,28,72
31,01,2,000
38,06,24,77
39,09,15,2011


In [23]:
# Verify date components
df_inconsistent_month = verify(df_inconsistent_month)

In [24]:
# Convert 'Year' column to numeric
df_inconsistent_month['Year'] = pd.to_numeric(df_inconsistent_month['Year'], errors='coerce')

In [25]:
# Save the extracted date components to the main dataframe
criteria = (df.Iter == 0) & (df_inconsistent_month['Year'].notna()) & (df_inconsistent_month['Year'] > 0)
df.loc[criteria, columns] = df_inconsistent_month[columns]
df.loc[criteria, 'Iter'] = iter + 1

info()

**Numer of rows with iter=1: 243**

,Code,Data,Iter,Day,Month,Year
3,1039963589,"October 7, 01 [report_end]",1,7,10,2001
9,1148116416,24 yo right handed woman with history of large right frontal mass s/p resection 11/3/1985 who had recent urgent R cranial wound revision and placement of L EVD for declining vision and increased drainage from craniotomy incision site and possible infection. She has a hx of secondary mania related to psychosis and manipulation of her right frontal lobe.,1,3,11,1985
10,1157934136,4/19/91 Communication with referring physician?: Not Done,1,19,4,1991
14,1191233809,07/29/1994 CPT code: 99203,1,29,07,1994
16,1218956332,6/10/72 SOS-10 Total Score:,1,10,6,1972


**Numer of rows with iter=0: 472**

,Code,Data,Iter,Day,Month,Year
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",0,0,0,0
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",0,0,0,0
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.",0,0,0,0
4,1048901075,"July, 4, 01 Primary Care Doctor:",0,0,0,0
5,1054311047,)and 8mo in 2009,0,0,0,0


In [26]:
columns_for_word_month = ['Day', 'Month', 'Year']

In [27]:
# Regular expression for dates with month as word and inconsistent order
regex_word_month = r"(\d{1,2})[-,\s]*(\w{3,})[-\s.]+(\d{2,4})"

In [28]:
# Apply the regex to extract date components
df_word_month = df[df.Iter == 0].Data.str.extract(regex_word_month)
df_word_month.dropna(inplace=True)
df_word_month.columns = columns_for_word_month

In [29]:
# Apply the conversion function to the 'Month' column
df_word_month['Month'] = df_word_month['Month'].apply(convert_word_month_to_number)

In [30]:
print(f"Number of rows matched = {df_word_month.shape[0]}")
df_word_month.head()

Number of rows matched = 169


,Day,Month,Year
0,12,11,16
12,15,09,91
15,14,01,1981
17,1,09,91
21,28,09,93


In [31]:
# Verify date components
df_word_month = verify(df_word_month)

In [32]:
# Convert 'Year' column to numeric
df_word_month['Year'] = pd.to_numeric(df_word_month['Year'], errors='coerce')

In [33]:
# Save the extracted date components to the main dataframe
criteria = (df.Iter == 0) & (df_word_month['Year'].notna()) & (df_word_month['Year'] > 0)
df.loc[criteria, columns] = df_word_month[columns]
df.loc[criteria, 'Iter'] = iter + 1

info()

**Numer of rows with iter=1: 412**

,Code,Data,Iter,Day,Month,Year
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",1,12,11,2016
3,1039963589,"October 7, 01 [report_end]",1,7,10,2001
9,1148116416,24 yo right handed woman with history of large right frontal mass s/p resection 11/3/1985 who had recent urgent R cranial wound revision and placement of L EVD for declining vision and increased drainage from craniotomy incision site and possible infection. She has a hx of secondary mania related to psychosis and manipulation of her right frontal lobe.,1,3,11,1985
10,1157934136,4/19/91 Communication with referring physician?: Not Done,1,19,4,1991
12,1164148605,stwin boys born 15 Sepember 91 Gambling behavior: No,1,15,09,1991


**Numer of rows with iter=0: 303**

,Code,Data,Iter,Day,Month,Year
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",0,0,0,0
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.",0,0,0,0
4,1048901075,"July, 4, 01 Primary Care Doctor:",0,0,0,0
5,1054311047,)and 8mo in 2009,0,0,0,0
6,1054668034,")HTN, hypercholesterolemia, DM, sleep apnea,, nephrolithiasis. chronic renal impairment, DVT since July 1977 on enoxaparin.",0,0,0,0


In [34]:
regex_with_ordinals = r"(\w{3,})\s+(\d{1,2})(?:st|nd|rd|th),\s+(\d{4})"

In [35]:
# Apply the regex to extract date components
df_with_ordinals = df[df.Iter == 0].Data.str.extract(regex_with_ordinals)
df_with_ordinals.dropna(inplace=True)
df_with_ordinals.columns = columns

In [36]:
print(f"Number of rows matched = {df_with_ordinals.shape[0]}")
df_with_ordinals.head()

Number of rows matched = 0


,Month,Day,Year


In [37]:
# Verify date components
df_with_ordinals = verify(df_with_ordinals)

In [38]:
# Convert 'Year' column to numeric
df_with_ordinals['Year'] = pd.to_numeric(df_with_ordinals['Year'], errors='coerce')

In [39]:
# Save the extracted date components to the main dataframe
criteria = (df.Iter == 0) & (df_with_ordinals['Year'].notna()) & (df_with_ordinals['Year'] > 0)
df.loc[criteria, columns] = df_with_ordinals[columns]
df.loc[criteria, 'Iter'] = iter + 1

info()

**Numer of rows with iter=1: 412**

,Code,Data,Iter,Day,Month,Year
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",1,12,11,2016
3,1039963589,"October 7, 01 [report_end]",1,7,10,2001
9,1148116416,24 yo right handed woman with history of large right frontal mass s/p resection 11/3/1985 who had recent urgent R cranial wound revision and placement of L EVD for declining vision and increased drainage from craniotomy incision site and possible infection. She has a hx of secondary mania related to psychosis and manipulation of her right frontal lobe.,1,3,11,1985
10,1157934136,4/19/91 Communication with referring physician?: Not Done,1,19,4,1991
12,1164148605,stwin boys born 15 Sepember 91 Gambling behavior: No,1,15,09,1991


**Numer of rows with iter=0: 303**

,Code,Data,Iter,Day,Month,Year
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",0,0,0,0
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.",0,0,0,0
4,1048901075,"July, 4, 01 Primary Care Doctor:",0,0,0,0
5,1054311047,)and 8mo in 2009,0,0,0,0
6,1054668034,")HTN, hypercholesterolemia, DM, sleep apnea,, nephrolithiasis. chronic renal impairment, DVT since July 1977 on enoxaparin.",0,0,0,0


In [40]:
regex_missing_day_month_word = r"([A-Za-z]{3,})\s+(\d{4})"

In [41]:
# Apply the regex to extract date components
df_with_missing_day_month_word = df[df.Iter == 0].Data.str.extract(regex_missing_day_month_word)
df_with_missing_day_month_word.dropna(inplace=True)
df_with_missing_day_month_word.columns = ['Month','Year']

In [42]:
# Apply the conversion function to the 'Month' column
df_with_missing_day_month_word['Month'] = df_with_missing_day_month_word['Month'].apply(convert_word_month_to_number)

In [43]:
print(f"Number of rows matched = {df_with_missing_day_month_word.shape[0]}")
df_with_missing_day_month_word.head()

Number of rows matched = 89


,Month,Year
6,07,1977
22,01,2007
29,09,1981
32,08,2012
36,09,1983


In [44]:
# Verify date components
df_with_missing_day_month_word = verify(df_with_missing_day_month_word)

In [45]:
# Convert 'Year' column to numeric
df_with_missing_day_month_word['Year'] = pd.to_numeric(df_with_missing_day_month_word['Year'], errors='coerce')

In [46]:
# Save the extracted date components to the main dataframe
criteria = (df.Iter == 0) & (df_with_missing_day_month_word['Year'].notna()) & (df_with_missing_day_month_word['Year'] > 0)
df.loc[criteria, columns] = df_with_missing_day_month_word[columns]
df.loc[criteria, 'Iter'] = iter + 1

info()

**Numer of rows with iter=1: 501**

,Code,Data,Iter,Day,Month,Year
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",1,12,11,2016
3,1039963589,"October 7, 01 [report_end]",1,7,10,2001
6,1054668034,")HTN, hypercholesterolemia, DM, sleep apnea,, nephrolithiasis. chronic renal impairment, DVT since July 1977 on enoxaparin.",1,1,07,1977
9,1148116416,24 yo right handed woman with history of large right frontal mass s/p resection 11/3/1985 who had recent urgent R cranial wound revision and placement of L EVD for declining vision and increased drainage from craniotomy incision site and possible infection. She has a hx of secondary mania related to psychosis and manipulation of her right frontal lobe.,1,3,11,1985
10,1157934136,4/19/91 Communication with referring physician?: Not Done,1,19,4,1991


**Numer of rows with iter=0: 214**

,Code,Data,Iter,Day,Month,Year
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",0,0,0,0
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.",0,0,0,0
4,1048901075,"July, 4, 01 Primary Care Doctor:",0,0,0,0
5,1054311047,)and 8mo in 2009,0,0,0,0
7,1082469285,"Septeber, 10, 70 CPT Code: 90792: With medical services",0,0,0,0


In [47]:
regex_missing_day_month_number = r"(\d{1,2})/(\d{4})"


In [48]:
# Apply the regex to extract date components
df_with_missing_day_month_number = df[df.Iter == 0].Data.str.extract(regex_missing_day_month_number)
df_with_missing_day_month_number.dropna(inplace=True)
df_with_missing_day_month_number.columns = ['Month','Year']

In [49]:
# Apply the conversion function to the 'Month' column
df_with_missing_day_month_number['Month'] = df_with_missing_day_month_number['Month'].apply(convert_word_month_to_number)

In [50]:
print(f"Number of rows matched = {df_with_missing_day_month_number.shape[0]}")
df_with_missing_day_month_number.head()

Number of rows matched = 105


,Month,Year
2,01,1974
8,01,2014
11,01,2003
13,01,1990
44,01,2006


In [51]:
# Verify date components
df_with_missing_day_month_number = verify(df_with_missing_day_month_number)

In [52]:
# Convert 'Year' column to numeric
df_with_missing_day_month_number['Year'] = pd.to_numeric(df_with_missing_day_month_number['Year'], errors='coerce')

In [53]:
# Save the extracted date components to the main dataframe
criteria = (df.Iter == 0) & (df_with_missing_day_month_number['Year'].notna()) & (df_with_missing_day_month_number['Year'] > 0)
df.loc[criteria, columns] = df_with_missing_day_month_number[columns]
df.loc[criteria, 'Iter'] = iter + 1

info()

**Numer of rows with iter=1: 606**

,Code,Data,Iter,Day,Month,Year
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",1,12,11,2016
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.",1,1,01,1974
3,1039963589,"October 7, 01 [report_end]",1,7,10,2001
6,1054668034,")HTN, hypercholesterolemia, DM, sleep apnea,, nephrolithiasis. chronic renal impairment, DVT since July 1977 on enoxaparin.",1,1,07,1977
8,1125769793,"Since 10/2014: Fatigued, more forgetful, impaired dexterity on her left hand. MRI reveals an approximately 4.2cm x 3.3cm x 2.5cm right parietal enhancing mass with surrounding edema",1,1,01,2014


**Numer of rows with iter=0: 109**

,Code,Data,Iter,Day,Month,Year
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",0,0,0,0
4,1048901075,"July, 4, 01 Primary Care Doctor:",0,0,0,0
5,1054311047,)and 8mo in 2009,0,0,0,0
7,1082469285,"Septeber, 10, 70 CPT Code: 90792: With medical services",0,0,0,0
24,1304079161,"Pt joined Army reserves in 2001 and has 3 years left in this commitment.-Mental Status Exam Was the exam performed? (If not, indicate reason): Yes",0,0,0,0


In [54]:
regex_missing_day_month = r"(\d{4})"

In [55]:
# Apply the regex to extract date components
df_with_missing_day_month = df[df.Iter == 0].Data.str.extract(regex_missing_day_month)
df_with_missing_day_month.dropna(inplace=True)
df_with_missing_day_month.columns = ['Year']

In [56]:
row_7 = df.iloc[7]  # Note: DataFrame index starts from 0, so row 7 corresponds to index 6
print(row_7)

Code                                                  1082469285
Raw      Septeber, 10, 70 CPT Code: 90792: With medical services
Data     Septeber, 10, 70 CPT Code: 90792: With medical services
Iter                                                           0
Day                                                            0
Month                                                          0
Year                                                           0
Date                                                           0
Name: 7, dtype: object


In [57]:
print(f"Number of rows matched = {df_with_missing_day_month.shape[0]}")
df_with_missing_day_month.head()

Number of rows matched = 57


,Year
5,2009
7,9079
24,2001
34,1975
59,1975


In [58]:
# Verify date components
df_with_missing_day_month = verify(df_with_missing_day_month)

In [59]:
# Convert 'Year' column to numeric
df_with_missing_day_month['Year'] = pd.to_numeric(df_with_missing_day_month['Year'], errors='coerce')

In [60]:
# Save the extracted date components to the main dataframe
criteria = (df.Iter == 0) & (df_with_missing_day_month['Year'].notna()) & (df_with_missing_day_month['Year'] > 0)
df.loc[criteria, columns] = df_with_missing_day_month[columns]
df.loc[criteria, 'Iter'] = iter + 1

info()

**Numer of rows with iter=1: 663**

,Code,Data,Iter,Day,Month,Year
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",1,12,11,2016
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.",1,1,01,1974
3,1039963589,"October 7, 01 [report_end]",1,7,10,2001
5,1054311047,)and 8mo in 2009,1,1,1,2009
6,1054668034,")HTN, hypercholesterolemia, DM, sleep apnea,, nephrolithiasis. chronic renal impairment, DVT since July 1977 on enoxaparin.",1,1,07,1977


**Numer of rows with iter=0: 52**

,Code,Data,Iter,Day,Month,Year
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",0,0,0,0
4,1048901075,"July, 4, 01 Primary Care Doctor:",0,0,0,0
55,1676425054,"n Abilify (added to Lexapro + Wellbutrin in Septmber, 5, 97)",0,0,0,0
83,2074807125,". Pt reports h/o difficulty with EtOH and opioids. EtOH: 1st use at age 10, regular use beginning in high school, more problematic at age 25 when started losing things. Drinking 12 beers/day over past year since returning to State College from Alaska. Unsure of heaviest use period. h/o frequent blackouts. Has passed out and woken up in variety of places. h/o withdrawal symptoms, 1st detox at age 25, no h/o withdrawal seizure, no DTs. opioids: 1st use at age 21 (perocet). Began using daily at age 25, at worst was using 8 to 10 percocets per day (depending on dose). Detox in 18, Septemer, 01, was using perioidically since back in State College, but has primarily been drinking. Last use 3 mths ago. no h/o suboxone or methadone. No h/o accidental OD. No h/o heroin use. No h/o IVDU. tobacco: 1 ppd since age 16, some interest in quitting, not right now. H/o quitting for a week.",0,0,0,0
89,2162384920,"Hep C and HIV negative, LFTs WNL (Augst, 11 05)Problems Opioid dependence",0,0,0,0


In [61]:
# Combine 'Day', 'Month', and 'Year' columns into a new 'Date' column
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']], errors='coerce')

In [62]:
df.head()

,Code,Raw,Data,Iter,Day,Month,Year,Date
0,1012720972,".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",".12, Noember 16- bad reaction to SpiceK2 - synthetic MJ- admitted to Crete Manor, Mcalester.",1,12,11,2016,2016-11-12
1,1039370009,".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",".April, 5 97: made a phone call to Mom and Mom commented that he was talking very fast, hard to interrupt, but was in super happy spirits, so didn't make a big deal of it.",0,0,0,0,NaT
2,1039574613,"A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.","A pleasant 28 yo woman with no formal psychiatric history and with a h/o SCCA of the right tongue (s/p partial glossectomy and neck dissection in 8/1974) referred to psycho-oncology for assistance with adjustment issues following recovery. The patient does not meet criteria for a major mood or anxiety disorder. She is not at imminent risk of harm to self or others. She would benefit from psychotherapy to help her integrate her experience of cancer and the break-up of her engagement, and to think through how to continue to create a life for herself moving forward.",1,1,01,1974,1974-01-01
3,1039963589,"October 7, 01 [report_end]","October 7, 01 [report_end]",1,7,10,2001,2001-10-07
4,1048901075,"July, 4, 01 Primary Care Doctor:","July, 4, 01 Primary Care Doctor:",0,0,0,0,NaT


In [64]:
# Sort DataFrame by 'Date' column in ascending order
df.sort_values(by=['Date', 'Raw'], inplace=True)

In [67]:
# Export DataFrame to a CSV file with specified fields
df.to_csv('output/results.csv', columns=['Code', 'Raw', 'Day', 'Month', 'Year', 'Date'], index=False)